In [6]:
%pip install torch torchvision torchaudio
%pip install opencv-python numpy mediapipe requests
%pip install transformers pyttsx3


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\Yahia\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\Yahia\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\Yahia\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [7]:
import cv2
import numpy as np
import math
import time
import requests
import mediapipe as mp

In [ ]:
def draw_text(image, text, position, font=cv2.FONT_HERSHEY_SIMPLEX, font_scale=0.5, color=(255, 255, 255), thickness=1):
    cv2.putText(image, text, position, font, font_scale, color, thickness, cv2.LINE_AA)

def draw_filled_rectangle(image, start_point, end_point, color):
    cv2.rectangle(image, start_point, end_point, color, cv2.FILLED)

def detect_fall(landmarks, image_height):
    left_shoulder_y = int(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y * image_height)
    left_hip_y = int(landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y * image_height)
    right_shoulder_y = int(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y * image_height)
    right_hip_y = int(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y * image_height)
    fall_threshold = 40 
    if left_hip_y - left_shoulder_y < fall_threshold or right_hip_y - right_shoulder_y < fall_threshold:
        return True
    else:
        return False
def calculate_angle(a,b,c):
    a=np.array(a)
    b=np.array(b)
    c=np.array(c)
    radians=np.arctan2(c[1]-b[1],c[0]-b[0])-np.arctan2(a[1]-b[1],a[0]-b[0])
    angle=np.abs(radians*108.0/np.pi)
    if angle >180.0:
        angle=360-angle
    return angle

def calculate_distance(point1, point2):
    return math.sqrt((point1.x - point2.x)**2 + (point1.y - point2.y)**2)

def normalize_distance(distance, reference_distance):
    return distance / reference_distance

import time

def distress_detection(landmarks, image, duration=10, print_interval=3):
    left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
    right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
    left_hand = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]
    right_hand = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]
    left_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]
    right_elbow = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value]

    shoulder_distance = calculate_distance(left_shoulder, right_shoulder)

    distance_left_hand_shoulder_normalized = normalize_distance(
        calculate_distance(left_hand, left_shoulder),
        shoulder_distance
    )
    distance_right_hand_shoulder_normalized = normalize_distance(
        calculate_distance(right_hand, right_shoulder),
        shoulder_distance
    )

    angle_left_arm = calculate_angle(
        [left_elbow.x, left_elbow.y],
        [left_hand.x, left_hand.y],
        [left_shoulder.x, left_shoulder.y]
    )

    angle_right_arm = calculate_angle(
        [right_elbow.x, right_elbow.y],
        [right_hand.x, right_hand.y],
        [right_shoulder.x, right_shoulder.y]
    )
    distress_flag = False
    current_time = time.time()
    if (
        (0.5 < distance_right_hand_shoulder_normalized < 1 and (110 < angle_right_arm < 145 or 60 < angle_left_arm < 100)) or
        (0.2 < distance_left_hand_shoulder_normalized < 0.8 and (110 < angle_right_arm < 145 or 60 < angle_left_arm < 100))
    ):
        distress_detection.last_print_time = current_time
        draw_text(image, "warning", (20, 100), color=(0, 0, 255))
        if not hasattr(distress_detection, 'start_time'):
            distress_detection.start_time = current_time
        elif current_time - distress_detection.start_time > duration:
            countdown = int(duration - (current_time - distress_detection.start_time))
            draw_text(image, str(countdown), (20, 70), color=(255, 0, 0))
            if countdown <= 10:
                draw_text(image, "danger", (20, 100), color=(255, 0, 0))
            if countdown <= 0:
                distress_flag = True
    return distress_flag

In [ ]:
from transformers import pipeline
import pyttsx3

text_generator = pipeline("text-generation", model="gpt2")

tts_engine = pyttsx3.init()
tts_engine.setProperty('rate', 150)
tts_engine.setProperty('volume', 1.0)

def generate_instructions():
    try:
        prompt = "A person has fallen and needs immediate help. Provide specific instructions."
        response = text_generator(prompt, max_length=50, num_return_sequences=1)
        instructions = response[0]["generated_text"].strip()
        return instructions
    except Exception as e:
        print(f"Error generating instructions: {e}")
        return "Please turn the person on their side and check for breathing."

def speak(text):
    tts_engine.say(text)
    tts_engine.runAndWait()

def provide_assistance():
    print("Generating assistance instructions...")
    instructions = generate_instructions()
    print(f"Instructions: {instructions}")
    speak(instructions)

Device set to use cpu


In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import time
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
distress_detection.start_time = None
cap = cv2.VideoCapture(1)

with mp_pose.Pose(min_detection_confidence=0.7, min_tracking_confidence=0.7) as pose:
    while cap.isOpened(): 
        ret, frame = cap.read()
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        try:
            landmarks = results.pose_landmarks.landmark
            left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
            left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
            distress_detection(landmarks, image)
            start_point = (5, 10)
            end_point = (400, 60)
            rectangle_color = (0, 0, 255)
            if detect_fall(landmarks, image.shape[0]):
                text = "Fallen"
                provide_assistance()
            else:
                left_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]
                shoulder_distance = calculate_distance(left_shoulder, right_shoulder)
                left_hand = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]
                right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
                text=str( angle_left_arm = calculate_angle(
        [left_elbow.x, left_elbow.y],
        [left_hand.x, left_hand.y],
        [left_shoulder.x, left_shoulder.y]
    ))
            draw_text(image, text, (20, 40), color=(255, 255, 255),font_scale=2.0)
        except Exception as e:
            pass 
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        cv2.imshow('Mediapipe Feed', image)
        if cv2.waitKey(10) & 0xFF == ord("q"): 
            break
cap.release()
cv2.destroyAllWindows()


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generating assistance instructions...
Instructions: A person has fallen and needs immediate help. Provide specific instructions. Provide the time at which the request to do so should be given, and the address. Provide information for individuals not in a medical emergency.

1. A person is required to
